<a href="https://colab.research.google.com/github/MaansAndersson/Krylov_solvers/blob/main/unicorn_minimial_mini_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
!dpkg --configure -a
!apt --fix-broken install python-pycurl python-apt

import platform, sys
python_version=platform.python_version()
from distutils.version import LooseVersion, StrictVersion
!pip install ideep4py
if ( LooseVersion(python_version) < LooseVersion("3.0.0")):
    print("Python3 is needed!");
    print("How to fix: Runtime/Change_runtime_type/Python 3")
    sys.exit()
    
try:
    !python3 -c 'from distutils.sysconfig import get_python_lib; print(get_python_lib(prefix="/usr/local"))'
    from dolfin import *; from mshr import *
except ImportError as e:
    !python3 -c 'from distutils.sysconfig import get_python_lib; print(get_python_lib(prefix="/usr/local"))'

    sys.path.append('/usr/lib/python3/dist-packages/')
    import sys, os; petsc_dir = '/usr/lib/petsc'; petsc_dir = os.path.join(os.getenv('PETSC_DIR') or petsc_dir, 'lib/python3/dist-packages'); petsc_dir in sys.path or sys.path.append(petsc_dir);
    #!apt-get install python3-dolfin
    !apt-get install -y -qq software-properties-common python-software-properties module-init-tools 
    !add-apt-repository -y ppa:fenics-packages/fenics #ppa:fenics-packages/fenics
    #!apt-get update -qq
    !apt install -y fenics #--no-install-recommends 
    !python3 -c 'from distutils.sysconfig import get_python_lib; print(get_python_lib(prefix="/usr/local"))'

    from dolfin import *; from mshr import *
    
import matplotlib.pyplot as plt;
from IPython.display import clear_output, display; import time; # import dolfin.common.plotting as fenicsplot 

import os, sys, shutil
import numpy as np 

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  import cupy as cp
  import scipy as sp
  from scipy.sparse import csr_matrix
  !git clone https://github.com/MaansAndersson/Krylov_solvers.git
  from Krylov_solvers import KLS 
  print(gpu_info)

dolfin_version = dolfin.__version__
print('dolfin version:', dolfin_version)

!rm -rf * # clean up all files
# Useful commands
# Remove an empty folder      : os.rmdir("my_results")
# Remove a folder with files  : shutil.rmtree("results")
# Make a folder               : os.mkdir("my_results")
# Runtime/Change_runtime_type/Python3

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Suggested packages:
  python-apt-dbg python-apt-doc libcurl4-gnutls-dev python-pycurl-dbg
  python-pycurl-doc
The following NEW packages will be installed:
  python-apt python-pycurl
0 upgraded, 2 newly installed, 0 to remove and 13 not upgraded.
Need to get 194 kB of archives.
After this operation, 873 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 python-apt amd64 1.6.5ubuntu0.5 [151 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 python-pycurl amd64 7.43.0.1-0.2 [43.1 kB]
Fetched 194 kB in 1s (321 kB/s)
Selecting previously unselected package python-apt.
(Reading database ... 146374 files and directories currently installed.)
Preparing to unpack .../python-apt_1.6.5ubuntu0.5_amd64.deb ...
Unpacking python-apt (1.6.5ubuntu0.5) ...
Selecting previously unselected package python-pycurl.
Preparing to unpack .../python-py

In [2]:
# Mesh from Fenics-HPC repo
#!curl -L https://kth.box.com/shared/static/qs0wzyfumua877wbpae71j7t0pm6a8c6 --output mesh0.xml


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100     7    0     7    0     0     10      0 --:--:-- --:--:-- --:--:--     0
100 1498k  100 1498k    0     0  1378k      0  0:00:01  0:00:01 --:--:-- 1378k


In [32]:
# Måns Andersson 2020
#

# Use compiler optimizations
parameters['form_compiler']['optimize'] = True
parameters['form_compiler']['cpp_optimize'] = True

#from mshr import *
from Krylov_solvers import KLS 
from dolfin import *
import math,sys


vtkfile_n = File('n.pvd')
vtkfile_u = File('u.pvd')
vtkfile_p = File('p.pvd')

stab ="GLS"       # GLS / SLD
TimeDep = True    # Depricated
Slip = True       # Standard formulation for slip 
NSlip = False     # Symmetric Nitsche's no-slip (or Slip)
FreeFlow = False  # \int p dx = 0 by h*h*q*p*dx 

## GEOMETRY
eps = 1e-6
lbfr = -10 
left = -0.5
right = 0.5 
rbfr = 30

## MESH ##
domain =  Cylinder(Point(lbfr,0,0),Point(rbfr, 0, 0), 10, 10) #Box(Point(lbfr,-10,-10),Point(rbfr,10,10))
box = Box(Point(-0.5,-0.5,-0.5),Point(0.5,0.5,0.5))
geometry = domain - box 
mesh = generate_mesh(geometry, 60)
#mesh = Mesh("mesh0.xml")

print(len(mesh.cells()), ' cells')
for i in range(0,2):
    cell_markers = MeshFunction("bool", mesh, mesh.topology().dim())
    for cell in cells(mesh):
      cell_markers[cell] = False
      p = cell.midpoint()
      #if (p.x()-30)**2+(p.y()-30)**2+(p.z()-2)**2 < 10*(1*i+1):
      #if (p.x()-22)**2+(p.y()-22)**2+(p.z()-2)**2 < 10*(1*i+1):
      if (p.x()-0)**2+(p.y()-0)**2+(p.z()-0)**2 < 2**2*0.7**i: #(10*(1*i+1))**2:
          cell_markers[cell] = True
    mesh = refine(mesh, cell_markers)
#mesh = (refine(mesh))
    print(len(mesh.cells()), ' cells')

dim = mesh.geometric_dimension()


pdeg = 1
V = VectorFunctionSpace(mesh, "CG", pdeg)
Q = FunctionSpace(mesh, "CG", pdeg)
Z = FunctionSpace(mesh, "DG", 0)
    
# SubDomains
class inflow(SubDomain):
    def inside(self, x, on_boundary):
        return x[0] < lbfr+eps and on_boundary
class outflow(SubDomain):
    def inside(self, x, on_boundary):
        return x[0] > rbfr-eps and on_boundary
class noslip_boundary(SubDomain):
    def inside(self, x, on_boundary):
        return  (x[0] <= rbfr-eps) and (x[0] > lbfr+eps) and on_boundary
class FreeFlowEXPRESSION(SubDomain):
    def inside(self, x, on_boundary):
        return (x[0]-0)**2+(x[1]-0)**2+(x[2]-0)**2 > 1**2 and on_boundary

# Boundary Conditions
#INFLOW_EXPRESSION = Expression(("(1-x[1]*x[1]-x[2]*x[2])","0","0"), degree = pdeg+1)
INFLOW_EXPRESSION = Expression(("1","0","0"), degree = pdeg+1)
bcm1 = DirichletBC(V, INFLOW_EXPRESSION, inflow()) #'on_boundary') #
bcm2 = DirichletBC(V, Expression(("0","0","0"), degree = pdeg+1), noslip_boundary())
bcm3 = DirichletBC(V, INFLOW_EXPRESSION, FreeFlowEXPRESSION()) #'on_boundary') #


# Strong bcs
if not FreeFlow:
  bcm = [bcm1] #, bcm2]
  bcc = DirichletBC(Q, Expression(("0"), degree = pdeg), outflow())
else: 
  bcm = [bcm3]
  bcc = []

# Slip marker
SlipMarker = Expression(("x[0]>lbufr+eps"), lbufr = lbfr, eps = eps, degree = 1)
#SlipMarker = Expression(("(x[0] > lbufr+eps)*(x[0] < rbufr-eps)*(abs(x[1]-y_max/2)<30)*(x[2]<z_max)"), lbufr = lbfr, eps = eps, rbufr = rbfr, y_max=0, z_max=0, degree = 1)
#SlipMarker = Expression(("(x[0] > lbufr+eps)*(x[0] < rbufr-eps)*(abs(x[1]-y_max/2)<30)*(x[2]<z_max)"), lbufr = lbfr, eps = eps, rbufr = rbfr, y_max=y_max, z_max=z_max, degree = 1)

# Define Functions
v = TestFunction(V)
q = TestFunction(Q)
u_ = TrialFunction(V)
p_ = TrialFunction(Q)
u = Function(V)
p = Function(Q)
u0 = Function(V)
p0 = Function(Q)
U = Function(V)
P = Function(Q)

""" Parameters """
hmin = mesh.hmin()
k = 1.*hmin #.25*(hmin)
nu = Constant(1/400000)
h = CellDiameter(mesh)
n = FacetNormal(mesh)
nn = Function(V)
maxit = 20
reltol = 0.05
d1 = 0.5  # Stabilisation momentum
d2 = 0.5  # Stabilisation continuity 

"""Weak Normal"""
sys = inner(u_,v)*dx(mesh) + 100/h*inner(u_+n,v)*ds(mesh)
a = lhs(sys)
b = rhs(sys)
A, B = assemble_system(a,b)
nn.vector()[:] = KLS.linear_solver(A, nn.vector()[:], B, 'BCG')


vtkfile_n << nn


t = 0 #initial time
if TimeDep:
    T = 10; #30;




""" Simulation """
j = 0
while ( t < T ):
  
  if(TimeDep):
    um = 0.5*(u + u0)
  else:
    um = u
    
    """ Stationary NS Equations """
  rm = (nu*inner(grad(um), grad(v)) + inner(grad(p) + grad(um)*um, v))*dx
  rc = div(um) * q * dx
    
  if ( TimeDep ):
    rm += inner(u - u0, v)/k*dx 
    rc += (2*k*inner(grad(p - p0), grad(q)))*dx #+ nu*q*p*dx # Second term??

  if ( FreeFlow ):
    rc += h*h*q*p*dx

  if (Slip) : 
    rm += SlipMarker*(1./h)*inner(um, nn)*inner(v, nn)*ds

  if (NSlip) :

    gamma1 = Constant(100) #No Slip
    gamma2 = Constant(100) #Slip 

    Ubc = (as_vector((0, 0, 0)))

    # Stokes Stokes Operator 
    # Already used in the formulation
    C_N = -nu*inner(dot(grad(um),nn), v)*ds(mesh)
    C_N_CO = inner(p*nn, v)*ds(mesh)
    
    # Skew symmetric Stokes 
    C_H_VL = -nu*inner(dot(grad(v),nn), um)*ds(mesh)
    C_H_VL_CO = -inner(q*nn, um)*ds(mesh)
    C_H_HL = -nu*inner(dot(grad(v),nn), Ubc)*ds(mesh)
    C_H_HL_CO = -inner(q*nn, Ubc)*ds(mesh)

    # Penalty terms
    PE_VL = SlipMarker*nu*gamma1/h*inner(um,v)*ds(mesh) + SlipMarker*gamma2/h*inner(dot(um,nn),dot(v,nn))*ds(mesh)
    PE_HL = SlipMarker*nu*gamma1/h*inner(Ubc,v)*ds(mesh) + SlipMarker*gamma2/h*inner(dot(Ubc,nn),dot(v,nn))*ds(mesh)

    # Should be zero due to Picard-iteration formulation
    zero_constant = Constant(0)

    rm += PE_VL - PE_HL
    rm += zero_constant*C_N_CO + C_H_VL - C_H_HL + C_N
    rc += zero_constant*C_H_VL_CO - C_H_HL_CO

  # NSE Stabilisation schemes
  if (stab == "GLS"): # Galerkin Least Square stabilisation
    rm += d1*h*(inner(grad(p) + grad(um)*um, grad(v)*um) + inner(div(um), div(v)))*dx
    rc += d2*h*inner(grad(p) + grad(um)*um, grad(q))*dx
  if (stab == "SLD"): #Streamline Diffusion stabilisation
    rm += d1*h*inner(grad(um)*um , grad(v)*um)*dx
    rc += d2*h*inner(grad(p),grad(q))*dx 

  """ Prepare Newton-Step """
  Jm = derivative(rm, u, u_)
  Jc = derivative(rc, p, p_)

  am = Jm
  Lm = action(Jm, u) - rm

  ac = Jc
  Lc = action(Jc, p) - rc


  for i in range(0, maxit):
    if True:
      Am, bm = assemble_system(am, Lm, bcm, form_compiler_parameters={'quadrature_degree': 2,
                                       'quadrature_scheme': 'vertex'})
      u.vector()[:] = KLS.linear_solver(Am, u.vector()[:], bm, 'BCG', 1e-8, 5000, timing=False, solver_info=True)
    if True:
      Ac, bc = assemble_system(ac, Lc, bcc, form_compiler_parameters={'quadrature_degree': 2,
                                       'quadrature_scheme': 'vertex'})
      p.vector()[:] = KLS.linear_solver(Ac, p.vector()[:], bc, 'CG', 1e-8, 5000, timing=False, solver_info=True)    

    umag = u.vector().norm("linf"); u0.vector().axpy(-1.0, u.vector())
    udiff = u0.vector().norm("linf"); relu = udiff / umag
    print("Step:", 'iteration: ', i ,"   realtive velocity update", relu)
    ## Check Convergance
    assign(u0, u); assign(p0, p)
    if relu < reltol:
      
      print("Time", t, "Newton iter", i)
      break
  assign(p0, p)
    
  #if TimeDep:
  assign(u0, u)
  t += k
  j += 1
  if TimeDep and 0 == np.mod(j,10): 
    print('Iteration: ', j, 'Time: ', t, 'Saved VTU-file', 0 == np.mod(j,10))
    vtkfile_u << u
    vtkfile_p << p


135505  cells
142526  cells
172123  cells
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
   Linear solver residual norm;  7.677231492512296e-09 7.677231566631181e-09 n of iterations:  66
Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
   Linear solver residual norm;  8.426366107911519e-09 n of iterations:  1084
Step: iteration:  0    realtive velocity update 1.0
   Linear solver residual norm;  8.646293872521544e-09 1.3893350974849168e-08 n of iterations:  933
   Linear solver residual norm;  9.393572304182934e-09 n of iterations:  1101
Step: iteration:  1    realtive velocity update 0.7707969985364764
   Linear solver residual norm;  4.804228170257641e-09 5.461307653812158e-09 n of iterations:  1069
   Linear solver residual norm;  9.450790048512307e-09 n of iterations:  1024
Step: iteration:  2    realtive velocity up

3351


In [30]:
print(len(p.vector()[:]))
try:
  from google.colab import files
except ImportError:
  pass
else:
  !mkdir SAVE_FOLDER
  !rm SAVE_FOLDER/*
  !mv  *pvd *vtu SAVE_FOLDER/
  !zip -r /content/file.zip /content/SAVE_FOLDER
  files.download("/content/file.zip")

26900
mkdir: cannot create directory ‘SAVE_FOLDER’: File exists
rm: cannot remove 'SAVE_FOLDER/*': No such file or directory
updating: content/SAVE_FOLDER/ (stored 0%)
updating: content/SAVE_FOLDER/u.pvd (deflated 76%)
updating: content/SAVE_FOLDER/p000004.vtu (deflated 64%)
updating: content/SAVE_FOLDER/u000003.vtu (deflated 63%)
updating: content/SAVE_FOLDER/u000001.vtu (deflated 63%)
updating: content/SAVE_FOLDER/u000006.vtu (deflated 63%)
updating: content/SAVE_FOLDER/u000005.vtu (deflated 63%)
updating: content/SAVE_FOLDER/u000007.vtu (deflated 63%)
updating: content/SAVE_FOLDER/p000006.vtu (deflated 64%)
updating: content/SAVE_FOLDER/u000000.vtu (deflated 63%)
updating: content/SAVE_FOLDER/p000000.vtu (deflated 64%)
updating: content/SAVE_FOLDER/p000001.vtu (deflated 64%)
updating: content/SAVE_FOLDER/n.pvd (deflated 28%)
updating: content/SAVE_FOLDER/p000007.vtu (deflated 64%)
updating: content/SAVE_FOLDER/p.pvd (deflated 76%)
updating: content/SAVE_FOLDER/u000004.vtu (deflated 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>